# Zephyr ADC Technical Report

## State Diagram

![state diagram](../state_diagram.png)

## Methods

Add more details / describe deviations where appropriate.

### Single-Channel ADC Sampling
* Quantify how linear the relationship is between the voltage applied to AIN0 and the LED1 blinking frequency.
  * Not sure how to quantify linearity?  Consider plotting the data and presenting the R2R^2R2 of a linear regression fit.
  * Remember that single data points for any single input voltage pair is not adequate; multiple measurements should be made and error bars presented on all plots.
* Quantify how linear the relationship is between the voltage applied to AIN1 and the LED1 ontime duration.

### Buffered Differential ADC Sampling
* Input a 500 Hz sinusoidal signal with a 1 V DC offset into the differential ADC input.
* Create a plot of your input signal and the buffered ADC samples (using the hex array output).  Discuss any differences between the input signal and your sampled signal.
* Compare your calculated RMS value to the expected RMS value of the sinusoidal signal.  Discuss any differences between your calculated RMS value and the expected RMS value.


## Results

Remember to read raw data in from a separate file!!

### Single-Channel ADC Sampling

#### Linearity AIN0 vs. LED1 Blinking Frequency

#### Linearity AIN1 vs. LED1 Ontime Duration

### Buffered Differential ADC Sampling

#### Input Signal vs. Buffered ADC Samples

#### RMS Value Comparison

## Discussion

How well did things agree with your nominal values?

If things didn't agree, why not?